## 3.2 字典的现代句法

### 3.2.1 字典推导式

In [2]:
dial_codes = [
    (880, 'Bangladesh'), 
    (55,  'Brazil'), 
    (86,  'China'), 
    (91,  'India'), 
    (62,  'Indonesia'), 
    (81,  'Japan'), 
    (234, 'Nigeria'), 
    (92,  'Pakistan'), 
    (7,   'Russia'), 
    (1,   'United States')
]
country_dial = {contry:code for code,contry in dial_codes}
print(country_dial)

{'Bangladesh': 880, 'Brazil': 55, 'China': 86, 'India': 91, 'Indonesia': 62, 'Japan': 81, 'Nigeria': 234, 'Pakistan': 92, 'Russia': 7, 'United States': 1}


### 3.2.2 映射拆包

In [3]:
def test(**kwargs):
    for key, value in kwargs.items():
        print(f"{key}: {value}")

In [4]:
test(**{"x": 1, "y": 2},z=3)

x: 1
y: 2
z: 3


In [5]:
a = {"x": 1, "y": 2,**{"z":3, "w": 4}}
print(a)

{'x': 1, 'y': 2, 'z': 3, 'w': 4}


### 3.2.3 使用 ｜ 合并映射

In [7]:
a = {"a": 1, "b": 2}
b = {"a": 1, "b": 5,"c": 3, "d": 4}
c = a|b
print(c)
d = b|a
print(d)

{'a': 1, 'b': 5, 'c': 3, 'd': 4}
{'a': 1, 'b': 2, 'c': 3, 'd': 4}


## 3.3 使用模式匹配处理映射

In [32]:
def match_dic_type(item: dict)->None:
    match item:
        case {"age": int(age)} if age < 28:
            print(f"too small")
        case {"name":str(name),"age":int(age),"other":dict(other)}:
            print(f"other is a dict with keys {other.keys()}")
        case {"name":str(name),"age":int(age),"other":list(other)}:
            print(f"other is a list with length {len(other)}")
        case {"name":str(name),"age":int(age)}:
            print(f"other is not a dict or list, name: {name}, age: {age}")
        case _:
            raise UserWarning("Unknown type of dict")

In [35]:
match_dic_type(dict(name="Alice", age=30, other={"key1": "value1", "key2": "value2"}))
match_dic_type(dict(name="Bob", age=25, other=["item1", "item2", "item3"]))
match_dic_type(dict(name="Charlie", age=40))
try:
    match_dic_type(dict(person="Dave", age=35, other="not a dict or list"))
except UserWarning as e:
    print(f"Caught an exception: {e}")

other is a dict with keys dict_keys(['key1', 'key2'])
too small
other is not a dict or list, name: Charlie, age: 40
Caught an exception: Unknown type of dict


## 3.4 映射类型的标准 API

### 3.4.1 "可哈希"指什么

- 如果一个对象的哈希码在整个生命周期内永不改变（依托__hash__()方法），而且可与其他对象比较（依托__eq__()方法），那么这个对象就是可哈希的。两个可哈希对象仅当哈希码相同时相等。

In [38]:
a = 10
print(hash(a))
b = "10"
print(hash(b))
c = (1,2,3)
print(hash(c))


10
-529462075801039706
529344067295497451


In [40]:
a = (1,2,(3,4,5))
print(hash(a))
try :
    b = (1,2,[3,4,5])
    print(hash(b))
except TypeError as e:
    print(f"Cannot hash a list: {e}")

2941677136697153642
Cannot hash a list: unhashable type: 'list'


### 3.4.2 常用映射方法概述

### 3.4.3 插入或更新可变的值

In [43]:
a = {"a":1}
try:
    print(a['b'])
except KeyError as e:
    print(f'KeyError:{e}')

KeyError:'b'


#### 用 get(key) 获取字典的值,KeyError错误

In [44]:
print(a.get('b', 'default_value'))  # Using get to avoid KeyError

default_value


#### setdefault()

In [46]:
a.setdefault('b',[1,2,3])
print(a)

{'a': 1, 'b': [1, 2, 3]}


## 3.5 自动处理缺失的键

### 3.5.1 defaultdict 处理缺失键的另一种选择

In [49]:
from collections import defaultdict
d = defaultdict(list)
d['a'].append(1)
d['a'].append(2)
print(d['a'])

[1, 2]


### 3.5.2  "__missing__" 方法

In [57]:
class MyDict(dict):
    def __missing__(self,key):
        if not isinstance(key,str):
            raise KeyError(f"Key {key} is not a string")
        self[key] = f"Default value for {key}"
        return self[key]
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default
a = MyDict(one=1, two=2)
print(a)

{'one': 1, 'two': 2}


In [58]:
a['three']=7
print(a)

{'one': 1, 'two': 2, 'three': 7}


In [61]:
a['four']
print(a)

{'one': 1, 'two': 2, 'three': 7, 'four': 'Default value for four'}


In [63]:
try:
    a[10]
except KeyError as e:
    print(f"KeyError: {e}")

KeyError: 'Key 10 is not a string'


### 3.5.3 标准库对__missing__方法的使用不一致

## 3.6 dict 的变体

### 3.6.1 collections.OrderedDict

### 3.6.2 collections.ChainMap